In [9]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1' #update to 1 gpu
# from data_pipeline import *
# from model_metrics import *
# from model_utils import *
import glob as glob
from fastai.vision import *
import seaborn as sns

torch.cuda.set_device(0)

In [2]:
train_root_dir = os.path.join(os.environ['JUPYTER_HOME'], 'datasets-ssd/SIIM_ACR/train')
csv_root = os.path.join(train_root_dir, 'train-rle.csv')
train_csv = pd.read_csv(csv_root)

print(train_root_dir)

/home/projects/21170165/datath03/jupyter/datasets-ssd/SIIM_ACR/train


In [5]:
data = (ImageList.from_csv('/','raid/projects/21170165/datath03/train.csv', folder=train_root_dir+'/images/1024/dicom',cols='ImageId',suffix='.png')
           .split_by_rand_pct(valid_pct=0.2, seed=34)
           .label_from_df(cols=' EncodedPixels')
           .databunch(bs=8, num_workers=0))

In [11]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy)

RuntimeError: CUDA error: out of memory

In [3]:
def custom_path_save(self, name:PathOrStr, path='./checkpoints', return_path:bool=False, with_opt:bool=True):
        "Save model and optimizer state (if `with_opt`) with `name` to `self.model_dir`."
        # delete #  path = self.path/self.model_dir/f'{name}.pth'
        # my addition: start
        if path=='': path = self.path/self.model_dir/f'{name}.pth'
        else: path = f'{path}/{name}.pth'
        # end
        if not with_opt: state = get_model(self.model).state_dict()
        else: state = {'model': get_model(self.model).state_dict(), 'opt':self.opt.state_dict()}
        torch.save(state, path)
        if return_path: return path

def custom_path_load(self, name:PathOrStr, path='./checkpoints', device:torch.device=None, strict:bool=True, with_opt:bool=None,purge=False):
        "Load model and optimizer state (if `with_opt`) `name` from `self.model_dir` using `device`."
        if device is None: device = self.data.device
        # delete # state = torch.load(self.path/self.model_dir/f'{name}.pth', map_location=device)
        # my addition: start
        if path=='': path = self.path/self.model_dir/f'{name}.pth'
        else: path = f'{path}/{name}.pth'
        state = torch.load(path, map_location=device) 
        # end
        if set(state.keys()) == {'model', 'opt'}:
            get_model(self.model).load_state_dict(state['model'], strict=strict)
            if ifnone(with_opt,True):
                if not hasattr(self, 'opt'): opt = self.create_opt(defaults.lr, self.wd)
                try:    self.opt.load_state_dict(state['opt'])
                except: pass
        else:
            if with_opt: warn("Saved filed doesn't contain an optimizer state.")
            get_model(self.model).load_state_dict(state, strict=strict)
        return self

learn.save = custom_path_save.__get__(learn)
learn.load = custom_path_load.__get__(learn)

NameError: name 'learn' is not defined

In [ ]:
learn.load('stage1',path=model_path)